# One-Class SVM(Support Vector Machine)

## What is SVM?
Support Vector Machine (SVM) is a supervised machine learning algorithm used for classification, regression, and outlier detection tasks. 

The key idea behind SVM is to find the optimal hyperplane that best separates the classes (categories) in the feature space. This hyperplane is chosen such that it maximizes the margin, which is the distance between the hyperplane and the nearest data points from each class, known as support vectors.

Here are some important concepts and characteristics of SVM:

1. **Hyperplane**: In a binary classification problem with two classes, a hyperplane is a decision boundary that separates the data points of one class from those of the other class. For datasets with more than two classes, SVM constructs multiple hyperplanes using various strategies (e.g., one-vs-one, one-vs-all).
1. **Margin**: The margin is the distance between the hyperplane and the nearest data points (support vectors) from each class. SVM aims to maximize this margin to achieve better generalization and improve the classifier's robustness to new data points.
1. **Kernel Trick**: SVM can handle nonlinear decision boundaries by using a kernel function to map the original input space into a higher-dimensional feature space where the classes are more easily separable. Common kernel functions include linear, polynomial, radial basis function (RBF), and sigmoid kernels.
1. **Regularization Parameter (C)**: SVM includes a regularization parameter (C) that controls the trade-off between maximizing the margin and minimizing the classification error. A small C value allows for a wider margin but may lead to misclassification of some training examples, while a large C value prioritizes correctly classifying all training examples but may result in a narrower margin.
1. **Support Vectors**: Support vectors are the the subset of training data points that closest to the decision boundary (hyperplane). These points influence the position and orientation of the hyperplane as the hyperplane is determined based on maximizing the margin between these support vectors.

SVM is widely used in various applications, including text categorization, image classification, bioinformatics, and financial forecasting. Its effectiveness, especially in high-dimensional spaces, combined with its ability to handle both linear and nonlinear decision boundaries, makes it a versatile and powerful machine learning algorithm.

## Why it called a Support Vector Machine?

The name "Support Vector Machine" emphasizes the algorithm's focus on identifying `support vectors` (critical data points) and constructing an optimal decision boundary (hyperplane) between different classes in the feature space. The term `"machine"` underscores the algorithm's role as a learning framework for building classification models based on training data.

## What is One-Class SVM?

It is a variant of SVM that is primarily used for outlier detection and novelty detection tasks. Unlike traditional SVM, which is typically used for binary classification tasks, One-Class SVM is designed to learn a representation of normal data points and then identify deviations from this representation as outliers.

The main idea behind One-Class SVM is to learn a decision boundary (hyperplane) that encapsulates the normal data points in the feature space. This decision boundary is constructed in such a way that it separates the normal data points from the origin or the center of the feature space while maximizing the margin around the normal data. Any data points that fall outside this boundary are considered outliers or anomalies.

Key characteristics of One-Class SVM include:

1. **Unsupervised Learning**: One-Class SVM is often used in an unsupervised learning setting, where only normal data points are available for training. The algorithm learns to distinguish between normal and abnormal data points based solely on the characteristics of the normal data.
1. **Single-Class Classification**: One-Class SVM performs single-class classification, where it learns to model only the distribution of normal data points without explicitly modeling other classes. It aims to identify deviations from the normal data distribution rather than distinguishing between multiple classes.
1. **Hyperparameter Tuning**: One-Class SVM includes hyperparameters such as the kernel type, regularization parameter (nu), and kernel parameters (if applicable), which need to be tuned to achieve optimal performance. The choice of hyperparameters can significantly affect the model's ability to detect outliers.
1. **Application in Outlier Detection**: One-Class SVM is commonly used for outlier detection tasks, where the goal is to identify data points that deviate significantly from the normal behavior of the dataset. It has applications in various domains, including fraud detection, intrusion detection, and anomaly detection in sensor data.

## Example

### How to re-train a One-Class SVM model using labeled data
It trained a OCSVM model, then add some labeled feedback data to train dataset to re-train the model.

In [13]:
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
import numpy as np

# Generate synthetic labeled data (normal and anomalous)
normal_data = np.random.normal(loc=0, scale=1, size=(1000, 2))  # Normal data
anomalous_data = np.random.uniform(low=-10, high=10, size=(50, 2))  # Anomalous data

# Label the data: 1 for normal, -1 for anomalous
normal_labels = np.ones(len(normal_data))
anomalous_labels = -np.ones(len(anomalous_data))

# Combine the labeled data
labeled_data = np.vstack([normal_data, anomalous_data])
labels = np.hstack([normal_labels, anomalous_labels])

# Split the labeled data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(labeled_data, labels, test_size=0.2, random_state=42)

# Instantiate and train the initial One-Class SVM model
ocsvm_model = OneClassSVM(nu=0.1)  # Adjust nu parameter as needed
ocsvm_model.fit(X_train)

# Evaluate initial model performance
initial_score = ocsvm_model.score_samples(X_test)
print("Initial Model Performance (Test Score):", initial_score.mean())

# Collect feedback data (additional labeled data)
feedback_data = np.random.normal(loc=2, scale=1, size=(100, 2))  # Additional normal feedback data
feedback_labels = np.ones(len(feedback_data))  # Label the feedback data as normal

# Combine the feedback data with the original training data
X_train_feedback = np.vstack([X_train, feedback_data])
y_train_feedback = np.hstack([y_train, feedback_labels])

# Re-train the One-Class SVM model with the feedback data
ocsvm_model.fit(X_train_feedback)

# Evaluate re-trained model performance
retrained_score = ocsvm_model.score_samples(X_test)
print("Retrained Model Performance (Test Score):", retrained_score.mean())

Initial Model Performance (Test Score): 12.646710130260601
Retrained Model Performance (Test Score): 13.60183430807445


In scikit-learn's OneClassSVM, the `score_samples()` method returns the "negative" distance to the decision function, which means that lower values indicate better (more normal) instances, and higher (less negative) values indicate more anomalous instances.

In other words:

* More negative scores (lower values) indicate that the data points are closer to the decision boundary and are considered more normal or inlier-like.
* Less negative or positive scores (higher values) indicate that the data points are further from the decision boundary and are considered more anomalous or outlier-like.

### Save and reload a trained scikit-learn model
We use the `joblib` library to serialize & deserialize scikit-learn objects in this example.

In [14]:
import joblib


# Assume you have already trained a OneClassSVM model
ocsvm_model_save = OneClassSVM(nu=0.1)  # Adjust nu parameter as needed
ocsvm_model_save.fit(X_train)

# Evaluate initial model performance
initial_score = ocsvm_model_save.score_samples(X_test)
print("Initial Model Performance (Test Score):", initial_score.mean())

# Save the trained model to a file
joblib.dump(ocsvm_model_save, 'ocsvm_model.pkl')

# Later, when you want to reload the model:
# Load the saved model from the file
ocsvm_model_reloaded = joblib.load('ocsvm_model.pkl')
ocsvm_model_reloaded.fit(X_train_feedback)

# Evaluate re-trained model performance
retrained_score = ocsvm_model.score_samples(X_test)
print("Retrained Model Performance (Test Score):", retrained_score.mean())

Initial Model Performance (Test Score): 12.646710130260601
Retrained Model Performance (Test Score): 13.60183430807445
